In [2]:
import pandas as pd
import json
from datetime import datetime

# Calibration

This notebook can be used to set the baseline parameters of the model. It generates the base parameter.json file that is used for all simulations.

## 1 Set the general simulation parameters 

In [3]:
TIME = 350
AGENTS = 100000
MONTE_CARLO_RUNS = 1
CITY = 'cape_town'

## 2 Set the start and end dates for the validation period

In [4]:
START_DATE = datetime.strptime('2020-03-29', '%Y-%m-%d')
END_DATE = datetime.strptime('2020-06-14', '%Y-%m-%d') 

## 3 Import data

### 3.1 Oxford Stringency Index

In [5]:
stringency_index = pd.read_csv('input_data/covid-stringency-index.csv')[pd.read_csv('input_data/covid-stringency-index.csv')['Code'] == 'ZAF']
stringency_index.index = [datetime.strptime(x, '%b %d, %Y') for x in stringency_index['Date']]
stringency_index = stringency_index['Government Response Stringency Index ((0 to 100, 100 = strictest))']

### 3.2 Google mobility data

In [6]:
mobility_data = pd.read_csv('input_data/Global_Mobility_Report.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')[pd.read_csv('input_data/Global_Mobility_Report.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')['country_region_code'] == 'ZA']
mobility_data = mobility_data[mobility_data['sub_region_1'] == 'Western Cape']
mobility_data.index = [datetime.strptime(x, '%Y-%m-%d') for x in mobility_data['date']]
mobility_data = mobility_data[mobility_data.columns[7:]].astype(float)

## 3 Set policy parameters for the duration of the simulation.
Policy parameters are input as a list that is as long as the simulation. This way they can change over the course of the simulation, in line with observed policy. 

The travel multiplier is set using the Google mobility data.

In [7]:
travel_multipliers = list(1 + mobility_data.mean(axis=1).loc[START_DATE:END_DATE] /100)

The maximum number of contacts allowed by the government is based on three trips in a mini bus taxi. A taxi holds 16 people in total (so 15 besides the driver). This is multiplied by 70% which the South African government stipulated was allowed. 

In [8]:
gathering_max_contacts = int(round((15 * 0.7 + 1))) * 3
gathering_max_contacts

36

In [9]:
lockdown_severeness = stringency_index.loc[START_DATE:END_DATE]
# omega_1 likelihood awareness
likelihood_awareness = [0.08 * (x / 100) for x in lockdown_severeness]
# omega_3 maximum contacts allowed
max_contacts = [round(gathering_max_contacts * (1 + (1 - (x / 100)))) for x in lockdown_severeness]
# omega_4 probability infection multiplier 1−(1−0.91)×0.66
inf_multiplier = [1 - ((1 - 0.93) * (x / 100)) for x in lockdown_severeness]  

Extend the policy parameters by assuming lockdown stringency on the last day remains constant for the rest of the simulation.

In [10]:
inf_multiplier = inf_multiplier + [inf_multiplier[-1] for x in range(len(lockdown_severeness), TIME)] # 0.31 was based on a study of face mask on hamsters by Yuen et al. (2020)
travel_multipliers = travel_multipliers + [travel_multipliers[-1] for x in range(len(lockdown_severeness), TIME)] 
likelihood_awareness = likelihood_awareness + [likelihood_awareness[-1] for x in range(len(lockdown_severeness), TIME)] 
max_contacts = max_contacts + [max_contacts[-1] for x in range(len(lockdown_severeness), TIME)] 

## 4 Set initial infections and age groups

Next, we assume that 14% of infections were detected at the start of the simulation and translate this to the initial number of cases at the start of the simulation. 

In [11]:
perc_infections_detects = 14
initial_agents = max(round((310 / (3740026 / AGENTS) * 100 / perc_infections_detects)), 20) # 310 cases / (population / agent) * 1 / 14% detected cases
initial_agents

59

The age groups are per decile. 

In [12]:
age_groups = ['age_0_10', 'age_10_20', 'age_20_30', 'age_30_40', 'age_40_50', 
              'age_50_60', 'age_60_70', 'age_70_80', 'age_80_plus']

## 5 Create the parameters 

In [13]:
parameters = {
    # Parameters related to model implementation
    "time": TIME, 
    "number_of_agents": AGENTS,
    "monte_carlo_runs": MONTE_CARLO_RUNS,
    
    # COVID-19 parameters (9)
    "exposed_days": 4, # average number of days without symptoms and being able to infect others
    "asymptom_days": 10, # average number of days agents are infected but do not have symptoms
    "symptom_days": 10,# average number of days agents have mild symptoms
    "critical_days": 8, # average number of days agents are in critical condition
    "probability_symptomatic": 0.6165, # determines whether an agent will become asymptomatic or asymptomatic spreader
    "no_hospital_multiplier": 1.79, # the increase in probability if a critical agent cannot go to the hospital SOURCE: Zhou et al. 2020
    "probability_transmission": 0.006, #0.005,#0.00335, # should be estimated to replicate realistic R0 number.
    "probability_critical": {key:value for key, value in zip(age_groups, [0.001, 0.003, 0.012, 0.032, 0.049, 0.102, 0.166, 0.244, 0.273])}, # probability that an agent enters a critical stage of the disease SOURCE: Verity et al.
    "probability_to_die": {key:value for key, value in zip(age_groups, [0.005, 0.021, 0.053, 0.126, 0.221, 0.303, 0.565, 0.653, 0.765])}, # probability to die per age group in critical stage SOURCE: Verity et al.
    
    # Cape Town specific parameters (1)
    "health_system_capacity": 0.0009179, # 3433 acute beds in CT / 3740026 population
    
    # Policy parameters
    # (1) physical distancing measures such as increased hygiëne & face mask adoption 
    "physical_distancing_multiplier": inf_multiplier, # 0.31 was based on a study of face mask on hamsters by Yuen et al. (2020)
    # (2) reducing travel e.g. by reducing it for work, school or all
    "visiting_recurring_contacts_multiplier": travel_multipliers,#[travel_multiplier for x in range(0, TIME)], # based on travel data
    # (3) Testing and general awareness
    'likelihood_awareness': likelihood_awareness, #li2020early this will be increased through testing, track & trace and coviid
    # (4) limiting mass contact e.g. forbidding large events, outside household. 
    "gathering_max_contacts": max_contacts, # based on the regulations for mini bus taxis --> (15 * 0.7) + driver
    
    # initial infections
    "total_initial_infections": [x for x in range(0, initial_agents)], # total agents infected in CT
    
    # additional parameter used to switch of informal districts
    "informality_dummy": 1.0, # setting this parameter at 0 will mean the lockdown is equally effective anywhere, alternative = 1
    
    # Technical parameters
    'init_infected_agent': 0, # to calculate R0
    "data_output": 'csv', # 'csv' or 'network', or 'False'
    
    # Depreciated paramters (can be used later)
    "probability_susceptible": 0.000, # probability that the agent will again be susceptible after having recovered
    "perc_infections_detects": perc_infections_detects / 100.0,
    'aware_status': ['i2'], # i1 can be added if there is large scale testing, this is optional
}

Next, we store these parameters in a .json file.

In [14]:
with open('parameters/parameters_{}.json'.format(CITY), 'w') as outfile:
    json.dump(parameters, outfile)